In [1]:
import pickle
import nltk
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import models

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from gensim.models import Phrases
from gensim.models.phrases import Phraser
from nltk.corpus import stopwords

C:\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df = pd.read_csv('datasheets/corpus_prepared.csv', sep='\t')
del df['Unnamed: 0']

In [3]:
texts = [list(map(lambda x: x.strip().strip('"').strip("'"), t.lstrip("['").rstrip("]").split(r"',"))) for t in df['text']]

In [80]:
# Create Dictionary
id2word = corpora.Dictionary(list(texts))

# Create Corpus
#texts = df['text']

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:10])
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)], [(7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)], [(15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)], [(1, 1), (6, 1), (17, 1), (22, 1)], [(4, 1), (8, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1)], [(8, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1)], [(8, 1), (37, 2), (38, 2), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)], [(50, 1), (51, 1), (52, 1), (53, 1), (54, 1)], [(37, 1), (38, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1)], [(28, 1), (37, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1)]]


[[('aim', 1),
  ('britain', 1),
  ('foremost', 1),
  ('learn', 1),
  ('make', 1),
  ('society', 1),
  ('world', 1)]]

In [82]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                            id2word=id2word, 
                                            num_topics=52, 
                                            update_every=1, 
                                            chunksize=1, 
                                            passes=1)

KeyboardInterrupt: 

In [46]:
#pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]

[(32,
  '0.183*"work" + 0.138*"basis" + 0.126*"act" + 0.068*"put" + 0.067*"follow" + '
  '0.059*"prosperity" + 0.055*"hospital" + 0.041*"elect" + 0.028*"institution" '
  '+ 0.015*"sentence"'),
 (49,
  '0.176*"year" + 0.169*"fully" + 0.125*"fund" + 0.074*"debt" + 0.055*"pay" + '
  '0.043*"force" + 0.042*"resourc" + 0.022*"could" + 0.021*"block" + '
  '0.020*"expand"'),
 (15,
  '0.130*"level" + 0.112*"possible" + 0.075*"vat" + 0.075*"fiscal" + '
  '0.054*"real" + 0.052*"structure" + 0.050*"restore" + 0.045*"project" + '
  '0.045*"currency" + 0.033*"regulate"'),
 (48,
  '0.225*"equality" + 0.101*"create" + 0.078*"reform" + 0.075*"job" + '
  '0.065*"committee" + 0.062*"essential" + 0.044*"demand" + 0.041*"transition" '
  '+ 0.040*"creation" + 0.033*"impact"'),
 (26,
  '0.401*"ensure" + 0.102*"current" + 0.044*"progressive" + '
  '0.037*"sustainable" + 0.037*"transparency" + 0.037*"forward" + '
  '0.027*"maintain" + 0.026*"skill" + 0.022*"corporation_tax" + '
  '0.022*"relief"'),
 (29,
  '0

In [47]:
lda_model.save('datasheets/LDA/lda.model')

In [4]:
model =  models.LdaModel.load('datasheets/LDA/lda.model')
pprint(model.print_topics())

[(17,
  '0.162*"citizen" + 0.119*"dail" + 0.088*"vote" + 0.068*"direct" + '
  '0.060*"mortgage" + 0.054*"transparent" + 0.053*"voluntary" + '
  '0.045*"barrier" + 0.040*"cap" + 0.030*"voting"'),
 (19,
  '0.193*"income" + 0.164*"plan" + 0.131*"north" + 0.052*"behaviour" + '
  '0.033*"previous" + 0.032*"british" + 0.032*"construction" + 0.025*"total" + '
  '0.023*"share" + 0.022*"figure"'),
 (14,
  '0.297*"state" + 0.126*"power" + 0.056*"hand" + 0.051*"control" + '
  '0.044*"question" + 0.029*"industry" + 0.027*"away" + 0.022*"reward" + '
  '0.021*"stop" + 0.019*"subsidy"'),
 (5,
  '0.153*"end" + 0.148*"bank" + 0.126*"disability" + 0.112*"scheme" + '
  '0.068*"particularly" + 0.050*"study" + 0.032*"abroad" + 0.025*"range" + '
  '0.025*"reach" + 0.025*"compulsory"'),
 (37,
  '0.224*"make" + 0.082*"legislation" + 0.068*"recommendation" + '
  '0.066*"review" + 0.061*"efficiency" + 0.052*"sense" + 0.042*"ombudsman" + '
  '0.041*"gender" + 0.031*"nama" + 0.031*"unprecedented"'),
 (9,
  '0.294

In [51]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpus, id2word)
vis

C:\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
39     0.035965 -0.027069       1        1  3.002581
18     0.039013 -0.087777       2        1  2.895639
30    -0.105032 -0.017408       3        1  2.648029
10     0.011281 -0.060271       4        1  2.421354
23    -0.048393  0.091557       5        1  2.349644
47     0.051984  0.070481       6        1  2.340075
43     0.173355  0.129884       7        1  2.281317
8      0.066208  0.058697       8        1  2.262275
1     -0.003306 -0.041889       9        1  2.117082
32     0.022558 -0.076321      10        1  2.092878
49     0.101710  0.035874      11        1  2.074015
28    -0.037711 -0.041245      12        1  2.022583
51     0.045226 -0.138921      13        1  2.015704
22     0.067946 -0.021938      14        1  1.993925
48    -0.025971  0.070451      15        1  1.983544
0      0.012545 -0.058989      16        1  1.977083
11    -0.011470 -0.062431      17        1  1.959164
40     0.080448  0.070769      18        1  1.958520
26     0.013724 -0.100645      19        1  1.957206
45     0.024915 -0.070660      20        1  1.934794
41    -0.055238  0.029677      21        1  1.933772
9      0.023312  0.055790      22        1  1.931959
4      0.059763  0.119145      23        1  1.907576
12    -0.132749 -0.008168      24        1  1.902811
6     -0.021566 -0.161983      25        1  1.895648
37    -0.135934  0.031488      26        1  1.891061
7     -0.037615  0.049578      27        1  1.883550
42     0.063082 -0.054021      28        1  1.878520
35     0.043990  0.001953      29        1  1.856119
27    -0.042188 -0.011876      30        1  1.851509
20    -0.126709  0.016732      31        1  1.836521
25     0.027025  0.034805      32        1  1.825901
14     0.003396  0.012831      33        1  1.823111
38    -0.012751  0.002410      34        1  1.816719
46     0.030468 -0.074117      35        1  1.800418
5      0.016428 -0.052078      36        1  1.784703
33    -0.103637 -0.010162      37        1  1.776941
15     0.054246 -0.065777      38        1  1.775526
21    -0.058755 -0.105081      39        1  1.774177
29     0.025891 -0.037404      40        1  1.757205
24     0.020796  0.100373      41        1  1.728408
34     0.090545  0.187936      42        1  1.672560
13     0.000982 -0.029259      43        1  1.656521
19     0.055150 -0.004067      44        1  1.637553
50     0.016051 -0.029773      45        1  1.636057
3     -0.125746  0.095838      46        1  1.634205
36     0.115650  0.085944      47        1  1.576135
16    -0.218036  0.066611      48        1  1.526284
44    -0.185250  0.094302      49        1  1.523330
17     0.043718 -0.062256      50        1  1.460795
31     0.036756 -0.055193      51        1  1.450402
2      0.013929  0.053648      52        1  1.306603, topic_info=      Category         Freq          Term        Total  loglift  logprob
term                                                                    
301    Default  2548.000000       include  2548.000000  30.0000  30.0000
53     Default  2289.000000      increase  2289.000000  29.0000  29.0000
94     Default  2196.000000        ensure  2196.000000  28.0000  28.0000
321    Default  2371.000000        people  2371.000000  27.0000  27.0000
1382   Default  2010.000000         irish  2010.000000  26.0000  26.0000
1378   Default  1994.000000       ireland  1994.000000  25.0000  25.0000
221    Default  2124.000000       service  2124.000000  24.0000  24.0000
149    Default  1902.000000       support  1902.000000  23.0000  23.0000
212    Default  1822.000000     community  1822.000000  22.0000  22.0000
254    Default  1591.000000           new  1591.000000  21.0000  21.0000
293    Default  1518.000000         state  1518.000000  20.0000  20.0000
106    Default  1554.000000        public  1554.000000  19.0000  19.0000
24     Default  1555.000000    government  1555.000000  18.0000  18.0000
